In [1]:
import pandas as pd
import numpy as np

# read all the 7 original excel files from the emails 

path = "C:/Users/xhoan/OneDrive/Desktop/ST542 Project Datasets from email/"

df1 = pd.read_excel(path + 'Creativity Assessment LSC 101 (Start of the Semester) 2021.Kosal.Kenny.xlsx')
df2 = pd.read_excel(path + 'Kennys.sections.Creativity.end of semester.survey.quiz.xlsx')
df3 = pd.read_excel(path + 'Creativity Assessment LSC 101 (End of the Semester.kosal.2021.xlsx')
df4 = pd.read_excel(path + 'Creativity Assessment LSC 101 (End of the Semester.Kosal.xlsx') # later, it is shown that we will not need this file
df5 = pd.read_excel(path + 'Creativity Assessment LSC 101 (End of the Semester.kosal.section12.xlsx') # later, it is shown that we will not need this file

df6 = pd.read_excel(path + 'TTCT scores with Fluency.xlsx') # we manually added the "SA Fluency" column from the physical non-digital copy
df7 = pd.read_excel(path + 'HS GPA.2021.and number match up.xlsx')

Cleaning Quiz and Survey Data (so, everything that has to do with df1, df2, df3, df4, df5)

In [2]:
# don't keep the row with averages in df1, df2, df3, df5
df1 = df1.iloc[:-2][:]
df2 = df2.iloc[:-2][:]
df3 = df3.iloc[:-2][:] 
df5 = df5.iloc[:-2][:] 

# rename "Score" column of df4 to match with that of df3
df4 = df4.rename(columns={"Score": "Score (out of 15)"})

In [3]:
# df5 is exactly the same as df3 and every row of df4 was in df3

kosal_end = df3

#from now on, we will work only with df1, df2, kosal_end to clean the quiz and survey data.

In [4]:
# create a list with those dataset names
list_qs = [df1, df2, kosal_end]

Cleaning df1 dataset

In [5]:
# Check if there is any student ID number that appears more than once in a dataset.
# If so, keep only the last row.
# It turned out that "200416174" appears twice in df1 - so the student took the quiz/survey twice in the beginning of the semester,
# and we keep the last submission. 

for x in list(df1['Student ID number'].unique()):
    df_x = df1[df1['Student ID number'] == x]
    if df_x.shape[0] != 1:
        df1.drop_duplicates(subset=['Student ID number'], keep='last', inplace=True, ignore_index=True)

In [6]:
# Check if there are any missing values in any column
df1.isnull().sum()

Timestamp                                            0
First Name                                           0
Last Name                                            0
Student ID number                                    0
LSC 101 Instructor                                   0
Intended Major                                       0
Zip Code of your Home Address (where you grew up)    0
Best Fit for How you Would Describe your Hometown    0
To which gender do you most identify?                0
Race                                                 0
S1_beginning                                         0
S2_beginning                                         0
S3_beginning                                         0
S4_beginning                                         0
S5_beginning                                         0
S6_beginning                                         0
S7_beginning                                         0
S8_beginning                                         0
S9_beginni

In [7]:
# Check the unique values of each column to see for any misalignment; we start first with 'First Name' column

# Some first names have an extra space at the end or at the beginning, so we remove it
for x in list(df1['First Name'].unique()):
    if x[-1] == ' ':  
        df1.loc[df1['First Name'] == x, 'First Name'] = x[:-1]
    if x[0] == ' ':
        df1.loc[df1['First Name'] == x, 'First Name'] = x[1:]
        
# Same issue with last names
for x in list(df1['Last Name'].unique()):
    if x[-1] == ' ':  
        df1.loc[df1['Last Name'] == x, 'Last Name'] = x[:-1]
    if x[0] == ' ':
        df1.loc[df1['Last Name'] == x, 'Last Name'] = x[1:]

# Convert all the first names and last names to proper case
df1["First Name"] = df1["First Name"].str.title()
df1["Last Name"] = df1["Last Name"].str.title()

In [8]:
# Convert all Student ID numbers into string for easy indexing
df1 = df1.astype({"Student ID number": str})

# Fix the "weird" student id numbers such as the one with a missing digit, or the one that was a first name by referring to dataset df3
df1.loc[df1['Student ID number'] == '20040939', 'Student ID number'] = '200409390'
df1.loc[df1['Student ID number'] == 'Sikuran', 'Student ID number'] ='200394691'

In [9]:
# "Zip Code" column in df1 has some weird values

# First, convert all zip codes into string for easy indexing
df1 = df1.astype({"Zip Code of your Home Address (where you grew up)": str})

# Fix the "weird" values by finding online the zip code that corresponds to the given street address, ... 

df1.loc[df1['Zip Code of your Home Address (where you grew up)'] == '5316 Teakwood Dr', 'Zip Code of your Home Address (where you grew up)'] = '28083'
#df1.loc[df1['Zip Code of your Home Address (where you grew up)'] == '28312/28348', 'Zip Code of your Home Address (where you grew up)'] = ''
df1.loc[df1['Zip Code of your Home Address (where you grew up)'] == '30068 (Atlanta, GA)', 'Zip Code of your Home Address (where you grew up)'] = '30068'
#df1.loc[df1['Zip Code of your Home Address (where you grew up)'] == '395009', 'Zip Code of your Home Address (where you grew up)'] = ''
df1.loc[df1['Zip Code of your Home Address (where you grew up)'] == '14201 Buckton Lane', 'Zip Code of your Home Address (where you grew up)'] = '28105'
#df1.loc[df1['Zip Code of your Home Address (where you grew up)'] == '85586 (Germany)', 'Zip Code of your Home Address (where you grew up)'] = ''

In [10]:
# Check if there is any student that appears more than once with different Student ID numbers

df1_name_unique = df1.groupby(['First Name','Last Name'])['Student ID number'].count()
if set(list(df1_name_unique)) == {1}:
    print("There is no student that appears with different ID numbers")
else:
    print("There are students that appear with different ID numbers")

There is no student that appears with different ID numbers


In [11]:
# Create a dictonary with unique value of some specific columns of df1
df1_dict = {'Intended Major': list(df1['Intended Major'].unique()),
      'Hometown': list(df1['Best Fit for How you Would Describe your Hometown'].unique()),
      'Gender' : list(df1['To which gender do you most identify?'].unique()),
      'Race': list(df1['Race'].unique())}

df1_dict

{'Intended Major': ['Zoology',
  'Biology - BS',
  'Biochemistry',
  'Nutrition',
  'Biology - BA',
  'Genetics',
  'A major outside of science',
  'Microbiology',
  'Plant Biology'],
 'Hometown': ['Small city',
  'Rural Area',
  'Large city',
  'Town',
  'Suburb',
  'Township',
  'Village'],
 'Gender': ['Male', 'Female', 'Non-Binary', 'Prefer not to answer'],
 'Race': ['Asian',
  'Caucasian; White',
  'African-American; Black; African',
  'Hispanic; Latinx',
  'Indian',
  'Multiracial',
  'Native Hawaiian or Other Pacific Islander']}

In [12]:
# Convert all answers of survey questions to integer
for c in list(df1.columns[10:23]):
    df1 = df1.astype({c: int})

Cleaning df2 dataset

In [13]:
# Check if there is any student that appears more than once with different Student ID numbers

df2_name_unique = df2.groupby(['Last Name'])['Student ID number'].count()

if set(list(df2_name_unique)) == {1}:
    print("There is no student that appears with different ID numbers")
else:
    print("There are students that appear with different ID numbers")

# These are the students, whose last name appears more than once and they have different Student ID number
for n in set(list(df2_name_unique)):
    if n!=1:
        print(df2_name_unique[df2_name_unique==n])

There are students that appear with different ID numbers
Last Name
Childers    2
Cook        2
Nguyen      2
Wright      2
Name: Student ID number, dtype: int64
Last Name
Smith    3
Name: Student ID number, dtype: int64


In [14]:
# Checked their corresponding data in df1 to see if those cases are by coincidence, or by mistake.
#It turned out that some were by coincidence and one by mistake.
# Fixing its Student ID number based on corresponding data in df1
df2.loc[df2['Student ID number'] == 20003, 'Student ID number'] = '200355513.0'

In [15]:
# Check if there is any student ID number that appears more than once in a dataset.
# If so, keep only the last row.

for x in list(df2['Student ID number'].unique()):
    df_x = df2[df2['Student ID number'] == x]
    if df_x.shape[0] != 1:
        df2.drop_duplicates(subset=['Student ID number'], keep='last', inplace=True, ignore_index=True)

In [16]:
# Check if there are any missing values in any column
df2.isnull().sum()

Timestamp                                            0
Email Address                                        0
Score on quiz                                        0
possible total of quiz                               0
Last Name                                            0
Student ID number                                    0
Intended Major                                       0
Zip Code of your Home Address (where you grew up)    0
Best Fit for How you Would Describe your Hometown    0
To which gender do you most identify?                0
Race                                                 0
S1_end                                               0
S2_end                                               0
S3_end                                               0
S4_end                                               0
S5_end                                               0
S6_end                                               0
S7_end                                               0
S8_end    

In [17]:
# Check the unique values of each column to see for any misalignment; we start first with 'Last Name' column

# Some last names have an extra space at the end, so we remove it
for x in list(df2['Last Name'].unique()):
    if x[-1] == ' ':  
        df2.loc[df2['Last Name'] == x, 'Last Name'] = x[:-1]
    if x[0] == ' ':
        df2.loc[df2['Last Name'] == x, 'Last Name'] = x[1:]
        
# Convert all the  last names to proper case
df2["Last Name"] = df2["Last Name"].str.title()

In [18]:
# Convert all Student ID numbers into string for easy indexing
df2 = df2.astype({"Student ID number": str})

# The ID's were showing as "200391289.0", so remove the last 2 characters
for x in list(df2['Student ID number'].unique()):
    df2.loc[df2['Student ID number'] == x, 'Student ID number'] = x[:-2]

# Checking if there is any "weird" value
for x in list(df2['Student ID number'].unique()):
    if len(x)!=9:
        print(x)
        
# Fix the "weird" student id numbers such as the one with 5 missing digits, or the one that had an extra digit by the corresponding data in df1
df2.loc[df2['Student ID number'] == '7449743329', 'Student ID number'] = '200399223'
df2.loc[df2['Student ID number'] == '2004', 'Student ID number'] ='200406241'

7449743329
2004


In [19]:
# Scroll the below list to check for any "weird" value
list(df2['Zip Code of your Home Address (where you grew up)'].unique())

[27406,
 13219,
 27880,
 27713,
 28315,
 18901,
 27370,
 '08055',
 22602,
 22152,
 28327,
 29466,
 28012,
 28374,
 28277,
 28105,
 15241,
 28601,
 30068,
 27587,
 60126,
 27518,
 '01516',
 28173,
 27592,
 29445,
 27830,
 '06443',
 27284,
 27539,
 28211,
 27522,
 27106,
 28152,
 28020,
 '07094',
 '77096, 07950, 27560',
 28806,
 28376,
 27591,
 28501,
 27312,
 23059,
 27851,
 27253,
 28306,
 28607,
 27012,
 27519,
 27524,
 27376,
 27616,
 28759,
 27249,
 '612 Fieldstone',
 28278,
 27410,
 28083,
 27983,
 27298,
 27502,
 13077,
 27530,
 28655,
 27407,
 27040,
 28269,
 28036,
 '10514 Greenhead View Rd',
 27707,
 27610,
 27320,
 27893,
 28539,
 27874,
 11740,
 27511,
 2111,
 27215,
 28144,
 27521,
 '2140w',
 49546,
 21102,
 '105 Wild Brook Ct. Cary, NC, 27519',
 27603,
 27571,
 27562,
 43015,
 27604,
 27043,
 27540,
 28328,
 28092,
 60564,
 28571,
 28659,
 27263,
 27103,
 10541,
 '129 Buckingham Place Rd',
 77406,
 27527,
 27870,
 28470,
 28352,
 14510,
 '5239 Jessup rd.',
 27520,
 28043,
 

In [20]:
# Create a dictonary with unique value of some specific columns of df2
df2_dict = {'Intended Major': list(df2['Intended Major'].unique()),
      'Hometown': list(df2['Best Fit for How you Would Describe your Hometown'].unique()),
      'Gender' : list(df2['To which gender do you most identify?'].unique()),
      'Race': list(df2['Race'].unique())}

df2_dict

{'Intended Major': ['Biology - BS',
  'Genetics',
  'Biology - BA',
  'Biochemistry',
  'Nutrition',
  'A major outside of science',
  'Zoology',
  'Plant Biology',
  'Microbiology'],
 'Hometown': ['Small city',
  'Suburb',
  'Rural Area',
  'Large city',
  'Town',
  'Township',
  'Village'],
 'Gender': ['Female', 'Male', 'Prefer not to answer', 'Non-Binary'],
 'Race': ['Caucasian; White',
  'African-American; Black; African',
  'Asian',
  'Hispanic; Latinx',
  'Indian',
  'Multiracial']}

In [21]:
# Convert all answers of survey questions to integer
for c in list(df2.columns[11:24]):
    df2 = df2.astype({c: int})

Cleaning kosal_end dataset

In [22]:
# Check if there is any student that appears more than once with different Student ID numbers

kosal_end_name_unique = kosal_end.groupby(['Last Name'])['Student ID number'].count()

if set(list(kosal_end_name_unique)) == {1}:
    print("There is no student that appears with different ID numbers")
else:
    print("There are students that appear with different ID numbers")

# These are the students, whose last name appears more than once and they have different Student ID number
for n in set(list(kosal_end_name_unique)):
    if n!=1:
        print(kosal_end_name_unique[kosal_end_name_unique==n])

There are students that appear with different ID numbers
Last Name
Best       2
Douglas    2
Henry      2
Johnson    2
Sanchez    2
Scott      2
Name: Student ID number, dtype: int64


In [23]:
kosal_end.loc[kosal_end['First Name'] == 'Cass', 'First Name'] = 'Cassandra'

In [24]:
# Check if there is any student ID number that appears more than once in a dataset.

if len(list(kosal_end['Student ID number'].unique())) == kosal_end.shape[0]:
    print("There's no ID that repeats")

There's no ID that repeats


In [25]:
# Check if there are any missing values in any column
kosal_end.isnull().sum()

Timestamp                                            0
Email Address                                        0
Score (out of 15)                                    0
First Name                                           0
Last Name                                            0
Student ID number                                    0
Intended Major                                       0
Zip Code of your Home Address (where you grew up)    0
Best Fit for How you Would Describe your Hometown    0
To which gender do you most identify?                0
Race                                                 0
S1_end                                               0
S2_end                                               0
S3_end                                               0
S4_end                                               0
S5_end                                               0
S6_end                                               0
S7_end                                               0
S8_end    

In [26]:
# Check the unique values of each column to see for any misalignment; we start first with 'First Name' column

# Some first names have an extra space at the end or at the beginning, so we remove it
for x in list(kosal_end['First Name'].unique()):
    if x[-1] == ' ':  
        kosal_end.loc[kosal_end['First Name'] == x, 'First Name'] = x[:-1]
    if x[0] == ' ':
        kosal_end.loc[kosal_end['First Name'] == x, 'First Name'] = x[1:]
        
# Same issue with last names
for x in list(kosal_end['Last Name'].unique()):
    if x[-1] == ' ':  
        kosal_end.loc[kosal_end['Last Name'] == x, 'Last Name'] = x[:-1]
    if x[0] == ' ':
        kosal_end.loc[kosal_end['Last Name'] == x, 'Last Name'] = x[1:]

# Convert all the first names and last names to proper case
kosal_end["First Name"] = kosal_end["First Name"].str.title()
kosal_end["Last Name"] = kosal_end["Last Name"].str.title()

In [27]:
# Convert all Student ID numbers into string for easy indexing
kosal_end = kosal_end.astype({"Student ID number": str})

# Fix the "weird" student id numbers
kosal_end.loc[kosal_end['Student ID number'] == 'i dont remember', 'Student ID number'] = '200405114'
kosal_end.loc[kosal_end['Student ID number'] == '2003433`4', 'Student ID number'] ='200343314'
kosal_end.loc[kosal_end['Student ID number'] == '7449743329', 'Student ID number'] = '200399223'
kosal_end.loc[kosal_end['Student ID number'] == '120', 'Student ID number'] = '200412052'
kosal_end.loc[kosal_end['Student ID number'] == '1517145', 'Student ID number'] = '200412189'

In [28]:
# Check if there is any student that appears more than once with different Student ID numbers

kosal_end_name_unique = kosal_end.groupby(['First Name','Last Name'])['Student ID number'].count()
if set(list(kosal_end_name_unique)) == {1}:
    print("There is no student that appears with different ID numbers")
else:
    print("There are students that appear with different ID numbers")

There is no student that appears with different ID numbers


In [29]:
# Create a dictonary with unique value of some specific columns of df1
kosal_end_dict = {'Intended Major': list(kosal_end['Intended Major'].unique()),
      'Hometown': list(kosal_end['Best Fit for How you Would Describe your Hometown'].unique()),
      'Gender' : list(kosal_end['To which gender do you most identify?'].unique()),
      'Race': list(kosal_end['Race'].unique())}

kosal_end_dict

{'Intended Major': ['Zoology',
  'Biology - BS',
  'A major outside of science',
  'Biochemistry',
  'Nutrition',
  'Genetics',
  'Biology - BA',
  'Microbiology',
  'Plant Biology'],
 'Hometown': ['Small city',
  'Town',
  'Rural Area',
  'Large city',
  'Suburb',
  'Village',
  'Township'],
 'Gender': ['Male', 'Female'],
 'Race': ['Asian',
  'Hispanic; Latinx',
  'Caucasian; White',
  'African-American; Black; African',
  'Indian',
  'Multiracial']}

In [30]:
# Convert all answers of survey questions to integer
for c in list(kosal_end.columns[11:24]):
    kosal_end = kosal_end.astype({c: int})

Cleaning df7 dataset

In [31]:
df7 = df7.astype({"Student ID": str})
df7 = df7.iloc[:,0:4]
df_gpa_id = df7[df7['Deidentified Number']!='XX']

Combining all the datasets to create a cleaned single one

In [32]:
# Dropping columns we don't need

df1 = df1[df1.columns[~df1.columns.isin(['Timestamp', 'Zip Code of your Home Address (where you grew up)'])]]
df2 = df2[df2.columns[~df2.columns.isin(['Timestamp', 'Email Address','possible total of quiz','Zip Code of your Home Address (where you grew up)'])]]
kosal_end = kosal_end[kosal_end.columns[~kosal_end.columns.isin(['Timestamp', 'Email Address','Zip Code of your Home Address (where you grew up)'])]]

In [33]:
# Specifying the professor in the data of the end of the semester

df2['LSC 101 Instructor'] = "Dr. Kenny Kuo"
kosal_end['LSC 101 Instructor'] = 'Dr. Erica Kosal'

In [34]:
# Find any students that is in both Dr. Kenny data and Dr. Kosal data
list1 = list(df2['Student ID number'].unique())
list2 = list(kosal_end['Student ID number'].unique())

intersection = list(set(list1).intersection(list2))

for s in intersection:
    kosal_end.drop(kosal_end.loc[kosal_end['Student ID number']==s].index, inplace=True)

In [35]:
# rename columns so that datasets have the ame column names
kosal_end = kosal_end.rename(columns={"Score (out of 15)": "Quiz Score",'Best Fit for How you Would Describe your Hometown':'Hometown Type','To which gender do you most identify?':'Gender'})
df2 = df2.rename(columns={"Score on quiz": "Quiz Score",'Best Fit for How you Would Describe your Hometown':"Hometown Type",'To which gender do you most identify?':'Gender'})
df1 = df1.rename(columns={'Best Fit for How you Would Describe your Hometown':"Hometown Type",'To which gender do you most identify?':'Gender'})

In [36]:
# Now that we have non-overlapping datasets df2 and kosal_end with same column names, we can combine them

kosal_end = kosal_end[kosal_end.columns[~kosal_end.columns.isin(['First Name'])]]
                                        
data_end = pd.concat([df2, kosal_end])                                        

In [37]:
# Unifying the last names for students that appear in df1 and data_end

df1.loc[df1['Student ID number'] == '200400013', 'First Name'] = 'Katherine'
df1.loc[df1['Student ID number'] == '200400013', 'Last Name'] = 'Angeles Solano'
df1.loc[df1['Student ID number'] == '200417044', 'Last Name'] = "D'Amaro"
df1.loc[df1['Student ID number'] == '200402287', 'Last Name'] = "Moreland-Holsomback"
df1.loc[df1['Student ID number'] == '200411439', 'Last Name'] = "Morenomoreno"

In [38]:
# Combining all the data about quiz and survey in data_qs dataset
data_qs = pd.merge(df1,data_end,on='Student ID number')

In [39]:
# Fixing the column in data_qs
data_qs = data_qs[data_qs.columns[~data_qs.columns.isin(['Last Name_y', 'LSC 101 Instructor_y'])]]

In [40]:
# Combining data_qs with GPA data
df_gpa_id = df_gpa_id.rename(columns={"Student ID": "Student ID number"})

data_qs_gpa = data_qs.merge(df_gpa_id, on='Student ID number', how='left')

In [41]:
df6 = df6.rename(columns={"Number": "Deidentified Number"})

# Drop some columns since we already have that information 
df6 = df6[df6.columns[~df6.columns.isin(['IntendedMajor', 'Hometown','Race','Gender'])]]

In [42]:
df6 = df6.astype({"Deidentified Number": str})
data_qs_gpa = data_qs_gpa.astype({"Deidentified Number": str})

In [43]:
# Combining TTCT score with data_qs_gpa
df = pd.merge(data_qs_gpa, df6, on='Deidentified Number', how='outer')

In [44]:
#df.to_excel("C:/Users/xhoan/OneDrive/Desktop/The Single Cleaned Dataset.xlsx")